In [1]:
import pandas as pd
import re
from ast import literal_eval
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file, datas
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
customer_id = '90'
formatted_attribute = 'volume_ml'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
curation_col = f'Q:{attribute}'
field1='Capacity (Volume) [Nom]'
fields = [field1]

In [3]:
df, custom_field_df=parameters(customer_id, formatted_attribute, field1)
# [*custom_field_df]
df_cur, df_custom=datas(df,custom_field_df, field1)

247
247


In [4]:
del df_cur['bucket_id']
del df_cur['product_id']
del df_cur['image_url']
del df_cur['value']

# Custom Fields

In [15]:
df_custom=custom_field(df_custom,field1,formatted_attribute)
pat=r'''(?i)(ml)|()'''
df_custom['match'] = df_custom[curation_col].apply(lambda x: re_extract(pat, x))
df_custom[curation_col]=df_custom[curation_col].apply(lambda x: re.sub(r'(?i)(?<=ml)\.','',str(x)))
df_custom=df_custom[df_custom['match'].astype(str)!='[]']

matchcustml=df_custom[['external_id',curation_col]]
print(len(matchcustml))
rounding(df_custom,curation_col, 'a-n-z')

8


buckets  bucket_id  product_id external_id  \
65  Correction Aids       8637   434481810  5637148298   
66  Correction Aids       8637   434481812  5637148300   
67  Correction Aids       8637   434481813  5637148301   
69  Correction Aids       8637   434481815  5637148308   
75  Correction Aids       8637   434487639  5637168447   
77  Correction Aids       8637   434490644  5637167980   
78  Correction Aids       8637   434490645  5637167983   
79  Correction Aids       8637   434490646  5637167987   

                                                                       product_name  \
65       BIC Wite-Out Extra Coverage Correction Fluid, 20 ml Bottle, White, 1/Dozen   
66            BIC Wite-Out Quick Dry Correction Fluid, 20 ml Bottle, White, 1/Dozen   
67             BIC Wite-Out Quick Dry Correction Fluid, 20 ml Bottle, White, 3/Pack   
69                        BIC Wite-Out Shake 'n Squeeze Correction Pen, 8 ml, White   
75                         Pentel Presto! Multipurpose Correction Pen, 12 ml, White   
77                            Paper Mate Liquid Paper Correction Pen, 6.8 ml, White   
78  Paper Mate Liquid Paper Fast Dry Correction Fluid, 22 ml Bottle, White, 1/Dozen   
79   Paper Mate Liquid Paper Fast Dry Correction Fluid, 22 ml Bottle, White, 3/Pack   

                                                                                                                                                                                                                                                                                                                                  long_desc  \
65                                                                    Wite-Out Extra Coverage Correction Fluid, 20 ml Bottle, White, 1/Dozen•Extra Coverage fluid covers the toughest mistakes, i.e. red ink and permanent marker.•Applies evenlydries to a satin smooth finish.•Covers photocopies, faxes, ink, permanent marker and more.   
66  Wite-Out Quick Dry Correction Fluid, 20 ml Bottle, White, 1/Dozen•Quick Dry fluid goes on easy and provides reduced dry time.•Covers photocopies, faxes, ink, permanent marker and more.•Advanced technology allows application of just the right amount of fluid every time!.•No fuss, no mess, just neater, more precise corrections.   
67   Wite-Out Quick Dry Correction Fluid, 20 ml Bottle, White, 3/Pack•Quick Dry fluid goes on easy and provides reduced dry time.•Covers photocopies, faxes, ink, permanent marker and more.•Advanced technology allows application of just the right amount of fluid every time!.•No fuss, no mess, just neater, more precise corrections.   
69                                                             Wite-Out Shake 'n Squeeze Correction Pen, 8 ml, White•Needle point tip for accurate, precise corrections.•Fast drying, premium coverage that dries to a smooth finish.•Soft squeeze barrel for increased control.•Covers photocopies, faxes, ink, permanent marker and more.   
75                                                                             Presto! Multipurpose Correction Pen, 12 ml, White•Opaque fluid lays down smoother, offers complete coverage with no show-through or flaking.•Fine point metal tip for is perfect for small print.•Pocket clip for on-the-go convenience.•Ozone-safe formula.   
77                                                                                                       Correction Pen, 6.8 ml, White•Double ball tip resists clogging and extends the life of the pen.•Precise tip is ideal for correcting individual letters and numbers.•Correction fluid is fast drying to allow for premium coverage.   
78                                                                                    Fast Dry Correction Fluid, 22 ml Bottle, White, 1/Dozen•Fast-drying fluid with excellent coverage.•Features foam wedge applicator for fast corrections.•Corrects ballpoint, gel, roller ball and most marker inks.•Smooth-apply foam wedge brush tip.   
79                       

# N/A Values

In [16]:
df_na=df_cur
regex_pattern_na=r'''(?i)((?<!\d)(?<!\w)(?<!\/)(?<!-)\.?(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)\s?ml)|(\bml\b)|((?<!Use\s\d\-)ml)|()'''
df_na['na_matches'] = df_na['long_desc'].apply(lambda x: re_extract(regex_pattern_na, x))

df_na=df_na[(df_na['na_matches'].astype(str)=='[]')]
df_na[curation_col]='n/a'
matchesnaml=df_na[['external_id',curation_col]]
print(len(matchesnaml))
df_na.head(5)

150


buckets external_id  \
1  Cleaners & Polishes  5637177057   
2  Cleaners & Polishes  5637192950   
3  Cleaners & Polishes  5637284098   
4  Cleaners & Polishes  5637330716   
5  Cleaners & Polishes  5637330717   

                                                                               product_name  \
1                                   Universal Dry Erase Cleaning Wet Wipes, 5 x 10, 50/Pack   
2                                               EXPO Dry Erase Surface Cleaner, 22oz Bottle   
3  AbilityOne 7930014541159 Pre-Moistened Whiteboard Wipes, 50/Dispenser Pack, 12 Packs/Box   
4                 Earth Friendly Products ECOS� PRO Biodegradable White Board Cleaner, 6/CT   
5                       Earth Friendly Products ECOS� PRO Biodegradable White Board Cleaner   

                                                                                                                                                                                                                                                long_desc  \
1                                                                                                             Dry Erase Cleaning Wet Wipes, 5 x 10, 50/Pack•Premoistened wipes in a handy pop-up container help keep dry erase surfaces looking like new.   
2                                               Dry Erase Surface Cleaner, 22oz Bottle•Nontoxic cleaner for whiteboards that is perfect for classroom and conference room use.•Cleans and refreshes whiteboards.•Simply spray and wipe with a soft cloth.   
3  7930014541159 Pre-Moistened Whiteboard Wipes, 50/Dispenser Pack, 12 Packs/Box•Soft and absorbent.•Treated with a special formula that cleans all white boards.•One-step cleaning.•Eliminates ghosting, shadowing, residues, stains and stubborn marks.   
4                                                                      ECOS� PRO Biodegradable White Board Cleaner, 6/CT•Restores white board surfaces to like-new condition without affecting surface for future use•Free of synthetic perfumes and dyes   
5                                                                            ECOS� PRO Biodegradable White Board Cleaner•Restores white board surfaces to like-new condition without affecting surface for future use•Free of synthetic perfumes and dyes   

                                                         url  \
1     https://wbmason.com/ProductDetail.aspx?ItemID=UNV43660   
2   https://wbmason.com/ProductDetail.aspx?ItemID=SAN1752229   
3   https://wbmason.com/ProductDetail.aspx?ItemID=NSN4541159   
4  https://wbmason.com/ProductDetail.aspx?ItemID=EFPPL986806   
5  https://wbmason.com/ProductDetail.aspx?ItemID=EFPPL986806   

  Capacity (Volume) [Nom] na_matches Q:volume_ml  
1                     NaN         []         n/a  
2                     NaN         []         n/a  
3                     NaN         []         n/a  
4                     NaN         []         n/a  
5                     NaN         []         n/a

# Unit Values

In [17]:
df_oz=df_cur[df_cur['na_matches'].astype(str)!='[]']
df_oz['na_matches']=df_oz['na_matches'].apply(lambda x: sorted(x)).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r'\,','","',str(x))).apply(lambda x: re.sub(r"-",' ',str(x))).apply(lambda x: re.sub(r'(?<!\s)gal',' gal',str(x))).apply(lambda x: re.sub(r'(?<!\d)\.','0.',str(x)))
curate(df_oz, 'na_matches', formatted_attribute, 'a-kn-z')

Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, url, Capacity (Volume) [Nom], na_matches, Q:volume_ml, rounding]
Index: []

In [18]:
matchml=df_custom[['external_id',curation_col]]
print(len(matchml))
rounding(matchml,curation_col, 'b-fh-km-z')

8


external_id Q:volume_ml rounding
65  5637148298       20 mL      [m]
66  5637148300       20 mL      [m]
67  5637148301       20 mL      [m]
69  5637148308        8 mL      [m]
75  5637168447       12 mL      [m]
77  5637167980      6.8 mL      [m]
78  5637167983       22 mL      [m]
79  5637167987       22 mL      [m]

In [19]:
matchoz

external_id Q:volume_ml rounding
65  5637148298       20 mL      [m]
66  5637148300       20 mL      [m]
67  5637148301       20 mL      [m]
69  5637148308        8 mL      [m]
75  5637168447       12 mL      [m]
77  5637167980      6.8 mL      [m]
78  5637167983       22 mL      [m]
79  5637167987       22 mL      [m]

In [21]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcust))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchml))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesnaml))

matchcust
matchml
matchesnaml


In [ ]:
stop

# send to the folder for upload

In [22]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, matchml)

In [23]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, matchcustml)

In [24]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, matchesnaml)